# Extracting Titanic Disaster Data From Kaggle

In [2]:
!pip install python-dotenv

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [3]:
from dotenv import load_dotenv, find_dotenv

In [4]:
#find .env by searching all dir
dotenv_path = find_dotenv()
print(dotenv_path)

C:\Users\oramo\Titanic\.env


In [5]:
load_dotenv(dotenv_path)

True

In [7]:
import os
KAGGAL_USERNAME = os.environ.get("KAGGAL_USERNAME")
KAGGAL_PASSWORD = os.environ.get("KAGGAL_PASSWORD")
print(KAGGAL_USERNAME)
#print(KAGGAL_PASSWORD)

ismailmohammad


In [2]:
#imports
import requests
from requests import Session
import os
from dotenv import load_dotenv, find_dotenv

In [3]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

#payload for post
payload = {
    'action': 'login',
    'username': os.environ.get("KAGGAL_USERNAME"),
    'password': os.environ.get("KAGGAL_PASSWORD")
}

#urls 
url_train = 'https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftrain.csv'
url_test = 'https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftest.csv'

# setup session
with Session() as c:
    c.post('https://www.kaggle.com/account/login?phase=emailSignIn&returnUrl=%2F', data=payload)
    response = c.get(url_test)
    #print(response)
    #print(response.text)
    #from IPython.core.display import display, HTML
    #display(HTML(response.text))


In [18]:
def extract_data(url, file_path):
    '''
    Extract data from kaggle
    '''
    with Session() as c:
        c.post('https://www.kaggle.com/account/login?phase=emailSignIn&returnUrl=%2F', data=payload)
        with open(file_path, 'w') as handle:
            response = c.get(url, stream=True)
            for block in response.iter_content(1024):
                handle.write(block)
        
    
        

In [20]:
#urls
url_train = 'https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftrain.csv'
url_test = 'https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftest.csv'

#file paths
raw_data_path = os.path.join(os.path.pardir,'data','raw')
print(raw_data_path)
train_data_path = os.path.join(raw_data_path,'train.csv')
test_data_path = os.path.join(raw_data_path,'test.csv')
print(train_data_path)
print(test_data_path)

#extract data
extract_data(url_train, train_data_path)
extract_data(url_test, test_data_path)




..\data\raw
..\data\raw\train.csv
..\data\raw\test.csv


In [23]:
!dir ..\data\raw

 Volume in drive C has no label.
 Volume Serial Number is AE35-1A70

 Directory of C:\Users\oramo\Titanic\data\raw

05/10/2020  12:53 AM    <DIR>          .
05/10/2020  12:53 AM    <DIR>          ..
01/09/2020  11:36 PM                 0 .gitkeep
05/10/2020  12:53 AM             9,340 test.csv
05/10/2020  12:53 AM             9,160 train.csv
               3 File(s)         18,500 bytes
               2 Dir(s)  478,226,784,256 bytes free


## Building the file script

In [26]:
get_raw_data_script_file = os.path.join(os.path.pardir,'src','data','get_raw_data.py')
print(get_raw_data_script_file)

..\src\data\get_raw_data.py


In [35]:
%%writefile $get_raw_data_script_file
import os
from dotenv import load_dotenv, find_dotenv
from requests import Session
import logging

#payload for post
payload = {
    'action': 'login',
    'username': os.environ.get("KAGGAL_USERNAME"),
    'password': os.environ.get("KAGGAL_PASSWORD")
}

def extract_data(url, file_path):
    '''
    Extract data from kaggle
    '''
    with Session() as c:
        c.post('https://www.kaggle.com/account/login?phase=emailSignIn&returnUrl=%2F', data=payload)
        with open(file_path, 'w') as handle:
            response = c.get(url, stream=True)
            for block in response.iter_content(1024):
                handle.write(block)

def main(project_dir):
    '''
    main method
    '''
    #get logging
    logger = logging.getLogger(__name__)
    logger.info('getting raw data')
    
    #urls 
    url_train = 'https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftrain.csv'
    url_test = 'https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftest.csv'

    #file paths
    raw_data_path = os.path.join(os.path.pardir,'data','raw')
    train_data_path = os.path.join(raw_data_path,'train.csv')
    test_data_path = os.path.join(raw_data_path,'test.csv')
    
    #extract data
    extract_data(url_train, train_data_path)
    extract_data(url_test, test_data_path)
    logging.info('downloading raw training and test data')

if __name__ == '__main__':
    #getting root dir 
    project_dir = os.path.join(os.path.dirname(__file__), os.pardir, os.pardir)
    print(__file__)
    print(__name__)
    #print(__main__)
    
    #setup logger
    log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_fmt)
    
    #find .env 
    dotenv_path = find_dotenv()
    #load .env
    load_dotenv(dotenv_path)
    
    #call the main
    main(project_dir)
    

Overwriting ..\src\data\get_raw_data.py


In [36]:
!python $get_raw_data_script_file

..\src\data\get_raw_data.py
__main__


2020-05-10 01:33:17,226 - __main__ - INFO - getting raw data
2020-05-10 01:33:17,579 - root - INFO - downloading raw training and test data
